<a href="https://colab.research.google.com/github/ankokovin/MyMLNotebooks/blob/master/MyCETINCalculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CETIN Calculator

## 1. Оценка функционального размера

In [0]:
#@title Ввод C,E,T,I и N { run: "auto", display-mode: "form" }
C = 11 #@param {type:"integer"}
E = 10 #@param {type:"integer"}
T = 10 #@param {type:"integer"}
I = 10 #@param {type:"integer"}
N = 6 #@param {type:"integer"}

CETIN = [C, E, T, I, N]
print(CETIN)

[11, 10, 10, 10, 6]


## 2. Расчёт базовой трудоёмкости

|#|Наименование процесса     |C        |E        |T         |I         |N       |
|-|------------------------------------------------|----------|---------|----------|---------|---------|
|1|Бизнес-моделирование        |32.12|28.33|0         |14.15|0        |
|2|Управление требованиями |58.03|28.04|0         |20.32|0        |
|3|Проектирование                     |45.42|61.75|31.35|37.52|24.02|
|4|Реализация                              |31.57|81.51|50.72|36.11|0         |
|5|Тестирование                          |88.96|0       |0         |0       |0         |
|6|Развёртывание                       |8.96 |0        |0         |0       |23.74 |


In [0]:
#@title norm - таблица выше { display-mode: "form" }
norm = [
[32.12, 28.33, 0    , 14.15, 0    ],
[58.03, 28.04, 0    , 20.32, 0    ],
[45.42, 61.75, 31.35, 37.52, 24.02],
[31.57, 81.51, 50.72, 36.11, 0    ],
[88.96, 0    , 0    , 0    , 0    ],
[8.96 , 0    , 0    , 0    , 23.74]
]
norm

[[32.12, 28.33, 0, 14.15, 0],
 [58.03, 28.04, 0, 20.32, 0],
 [45.42, 61.75, 31.35, 37.52, 24.02],
 [31.57, 81.51, 50.72, 36.11, 0],
 [88.96, 0, 0, 0, 0],
 [8.96, 0, 0, 0, 23.74]]

$S_i = [C*S_i(C) + E*N_i(E) + T * N_i(T) + I*N_i(I) + N * S_i(N)]/165$

или по-русски (по матричному)

$S =  \begin{pmatrix}
  32.12&& 28.33&& 0      && 14.15&& 0        \\
  58.03&& 28.04&& 0      && 20.32&& 0        \\
  45.42&& 61.75&& 31.35&& 37.52&& 24.02 \\
  31.57&& 81.51&& 50.72&& 36.11&& 0        \\
  88.96&& 0      && 0       && 0      && 0        \\
  8.96  && 0      && 0       && 0      && 23.74
 \end{pmatrix} *
 \begin{pmatrix}
  C  \\
  E  \\
  T \\
  I \\
  N
 \end{pmatrix}*\frac{1}{165}$
 
 $S = norm*CETIN^T*\frac{1}{165}$

In [0]:
#@title Расчёт по формуле выше { display-mode: "form" }
import numpy as np

S = np.transpose( np.dot(np.matrix(norm), np.transpose( np.matrix(CETIN))))/165
print(S)

[[ 4.71587879  6.79957576 11.81781818 12.30709091  5.93066667  1.46060606]]


## 3. Определение поправочных коэффициентов

In [0]:
#@title Определение частных поправочных коэффициентов { run: "auto", display-mode: "form" }

#@markdown ## Внутренние факторы

#@markdown ### K1 Режим эксплуатации ИС
#@markdown Определяется в зависимости от конкретных технологий или типов обработки, принятых в системе программного обеспечения. Принимает следующие значения:
#@markdown - 1 - обработка данных в режиме разделения времени
#@markdown - 1,04 - параллельная обработка данных
#@markdown - 1,05 - обработка данных в режиме реального времени
#@markdown - 1,07 - совмещенная обработка данных
K1 = 1.05 #@param ["1", "1.04", "1.05", "1.07"] {type:"raw"}
#@markdown Обоснование: 
K1_reason = "\u041F\u043E\u0441\u043A\u043E\u043B\u044C\u043A\u0443 \u0437\u0430\u043F\u0440\u043E\u0441\u044B \u043D\u0430 \u043F\u043E\u043B\u0443\u0447\u0435\u043D\u0438\u0435 \u043A\u043E\u043D\u0442\u0435\u043D\u0442\u0430 \u0434\u043E\u043B\u0436\u043D\u044B \u0431\u044B\u0442\u044C \u043E\u0431\u0440\u0430\u0431\u043E\u0442\u0430\u043D\u044B \u043A\u0430\u043A \u043C\u043E\u0436\u043D\u043E \u0431\u044B\u0441\u0442\u0440\u0435\u0435, \u0431\u0443\u0434\u0435\u043C \u0441\u0447\u0438\u0442\u0430\u0442\u044C, \u0447\u0442\u043E \u0418\u0421 \u0440\u0430\u0431\u043E\u0442\u0430\u0435\u0442 \u0432 \u0440\u0435\u0436\u0438\u043C\u0435 \u0440\u0435\u0430\u043B\u044C\u043D\u043E\u0433\u043E \u0432\u0440\u0435\u043C\u0435\u043D\u0438 \u0434\u043B\u044F \u043E\u0431\u0435\u0441\u043F\u0435\u0447\u0435\u043D\u0438\u044F \u043E\u0442\u0432\u0435\u0442\u0430 \u0434\u0430\u043D\u043D\u044B\u043C \u0437\u0430\u043F\u0440\u043E\u0441\u0430\u043C." #@param {type:"string"}

#@markdown ### K2 Масштаб ИС
#@markdown Масштаб может быть определен количеством одновременно работающих пользователей. Принимает следующие значения: Принимает следующие значения:
#@markdown - 0,95  малые ИС (до 10 пользователей с непродолжительным ЖЦ)
#@markdown - 1 - средние ИС (от 11 до 100 пользователей с длительным ЖЦ с возможностью роста до крупных систем)
#@markdown - 1,05 - крупные ИС (от 101 до 1000 пользователей с длительным ЖЦ и миграцией унаследованных систем)
#@markdown - 1,08 - сверхбольшие (свыше 1000 пользователей)
K2 = 1.05 #@param ["0.95", "1", "1.05", "1.08"] {type:"raw"}
#@markdown Обоснование: 
K2_reason = "\u041A \u0441\u043E\u0436\u0430\u043B\u0435\u043D\u0438\u044E, \u043C\u0430\u0440\u043A\u0435\u0442\u0438\u043D\u0433\u043E\u0432\u044B\u0445 \u0438\u0441\u0441\u043B\u0435\u0434\u043E\u0432\u0430\u043D\u0438\u0439 \u0438 \u043F\u0440\u043E\u0433\u043D\u043E\u0437\u043E\u0432, \u043A\u0430\u043A\u043E\u0435 \u043A\u043E\u043B\u0438\u0447\u0435\u0441\u0442\u0432\u043E \u0430\u0443\u0434\u0438\u0442\u043E\u0440\u0438\u0438 \u043C\u044B \u0441\u043C\u043E\u0436\u0435\u043C \u043F\u0440\u0438\u0432\u043B\u0435\u0447\u044C, \u043F\u043E\u043A\u0430 \u043D\u0435 \u0431\u044B\u043B\u043E \u043F\u0440\u043E\u0438\u0437\u0432\u0435\u0434\u0435\u043D\u043E, \u0434\u0430\u043D\u043D\u0430\u044F \u043E\u0446\u0435\u043D\u043A\u0430 \u0431\u044B\u043B\u0430 \u0441\u0434\u0435\u043B\u0430\u043D\u0430 \u043F\u043E \u043F\u0440\u0438\u043D\u0446\u0438\u043F\u0443 \"\u043F\u0430\u043B\u044C\u0446\u0435\u043C \u0432 \u043D\u0435\u0431\u043E\" \u0438 \u043F\u043E\u0434\u0432\u0435\u0440\u0436\u0435\u043D\u0430 \u043A\u043E\u0440\u0440\u0435\u043A\u0442\u0438\u0440\u043E\u0432\u043A\u0435." #@param {type:"string"}

#@markdown ### K3 Стабильность ИС 
#@markdown Определяется в зависимости от ее внутренних эволюционных аспектов или стабильности в процессе сопровождения. Принимает следующие значения:
#@markdown - 1,1 - постоянное внесение изменений
#@markdown - 1 - дискретное внесение изменений
#@markdown - 0,95 - маловероятное внесение изменений
K3 = 1 #@param ["1.1","1","0.95"]{type:"raw"}
#@markdown Обоснование: 
K3_reason = "\u041E\u0431\u043D\u043E\u0432\u043B\u0435\u043D\u0438\u044F \u044F\u0432\u043B\u044F\u044E\u0442\u0441\u044F \u0434\u043E\u0441\u0442\u0430\u0442\u043E\u0447\u043D\u043E \u0432\u0435\u0440\u043E\u044F\u0442\u043D\u044B\u043C\u0438 \u0441\u043E\u0431\u044B\u0442\u0438\u044F\u043C\u0438, \u043D\u043E \u043D\u0435\u043E\u0431\u0445\u043E\u0434\u0438\u043C\u043E\u0441\u0442\u044C \u0432\u044B\u043F\u043E\u043B\u043D\u044F\u0442\u044C \u0438\u0445 \u043F\u043E\u0441\u0442\u043E\u044F\u043D\u043D\u043E \u043E\u0442\u0441\u0443\u0442\u0441\u0442\u0432\u0443\u0435\u0442." #@param {type:"string"}

#@markdown ### K4 Защита от несанкционированного доступа
#@markdown Предотвращение или существенное затруднение несанкционированного доступа. Принимает следующие значения:
#@markdown - 1,05 - сильная 
#@markdown - 1 - средняя
#@markdown - 0,98 - слабая
K4 = 1.05 #@param ["1.05", "1", "0.98"]{type:"raw"}
#@markdown Обоснование: 
K4_reason = "\u041D\u0430 \u0442\u043E \u0435\u0441\u0442\u044C \u043D\u0435\u0441\u043A\u043E\u043B\u044C\u043A\u043E \u043F\u0440\u0438\u0447\u0438\u043D:  \u041D\u0435\u043E\u0431\u0445\u043E\u0434\u0438\u043C\u043E \u0437\u0430\u0449\u0438\u0442\u0438\u0442\u044C \u043A\u043E\u043C\u043C\u0435\u0440\u0447\u0435\u0441\u043A\u0443\u044E \u0438\u043D\u0444\u043E\u0440\u043C\u0430\u0446\u0438\u044E - \u043F\u0440\u043E \u043C\u043E\u043D\u0435\u0442\u0438\u0437\u0430\u0446\u0438\u044E. \u041F\u043E\u0441\u043A\u043E\u043B\u044C\u043A\u0443 \u043F\u0440\u043E\u0435\u043A\u0442 \u0438\u043C\u0435\u0435\u0442 \u043D\u0430\u043F\u0440\u0430\u0432\u043B\u0435\u043D\u043D\u043E\u0441\u0442\u044C \u043D\u0430 IT, \u0438\u043C\u0435\u0435\u0442\u0441\u044F \u0432\u0435\u0440\u043E\u044F\u0442\u043D\u043E\u0441\u0442\u044C, \u0447\u0442\u043E \u043A\u0430\u043A\u043E\u0439-\u043D\u0438\u0431\u0443\u0434\u044C \u0445\u0430\u043A\u0435\u0440 \u0432\u043E\u0441\u043F\u0440\u0438\u043C\u0435\u0442 \u044D\u0442\u043E \u043A\u0430\u043A \"\u0432\u044B\u0437\u043E\u0432\". \u041A\u0442\u043E \u0437\u043D\u0430\u0435\u0442, \u043A\u0430\u043A\u0438\u0435 \u043E\u0442\u043D\u043E\u0448\u0435\u043D\u0438\u044F \u043C\u043E\u0433\u0443\u0442 \u043F\u043E\u043B\u0443\u0447\u0438\u0442\u044C\u0441\u044F \u0441 \u043D\u0430\u0451\u043C\u043D\u043E\u0439 \u043A\u043E\u043C\u0430\u043D\u0434\u043E\u0439 \u0440\u0430\u0437\u0440\u0430\u0431\u043E\u0442\u043A\u0438..." #@param {type:"string"}

#@markdown ### K5 Защита программ и данных (на уровне ОС, на уровне сетевого ПО, на уровне СУБД)
#@markdown - 1,03 - сильная
#@markdown - 1 - средняя
#@markdown - 0,97 - слабая
K5 = 1.03 #@param ["1.03", "1", "0.97"]{type:"raw"}
#@markdown Обоснование: 
K5_reason = "\u0410\u043D\u0430\u043B\u043E\u0433\u0438\u0447\u043D\u043E \u043F\u0440\u0435\u0434\u044B\u0434\u0443\u0449\u0435\u043C\u0443" #@param {type:"string"}

#@markdown ### K6 Контрольный след операций
#@markdown Возможность фиксации несанкционированных изменений в системе:
#@markdown - 1 - не имеется 
#@markdown - 1,08 - выборочное отслеживание 
#@markdown - 1,13 - полное отслеживание 
K6 = 1.08 #@param ["1", "1.08", "1.13"]{type:"raw"}
#@markdown Обоснование: 
K6_reason = "\u0422\u043E\u0442\u0430\u043B\u044C\u043D\u044B\u0439 \u043A\u043E\u043D\u0442\u0440\u043E\u043B\u044C \u043D\u0430\u0434 \u0441\u0438\u0441\u0442\u0435\u043C\u043E\u0439 \u043D\u0435 \u0442\u0440\u0435\u0431\u0443\u0435\u0442\u0441\u044F, \u043D\u043E \u0432\u043E\u0437\u043C\u043E\u0436\u043D\u043E\u0441\u0442\u044C \u043E\u0442\u0441\u043B\u0435\u0434\u0438\u0442\u044C \u0432\u044B\u0431\u043E\u0440\u043E\u0447\u043D\u043E - \u0442\u0440\u0435\u0431\u0443\u0435\u0442\u0441\u044F \u0434\u043B\u044F \u0431\u0435\u0437\u043E\u043F\u0430\u0441\u043D\u043E\u0441\u0442\u0438." #@param {type:"string"}

#@markdown ### К7 Отказоустойчивость
#@markdown Свойство системы непрерывно сохранять работоспособное состояние в течении некоторого времени. Принимает следующие значения:
#@markdown - 1,15 - Высокая 
#@markdown - 1 - Средняя
#@markdown - 0,92 - Низкая
K7 = 1 #@param ["1.15", "1", "0.92"]{type:"raw"}
#@markdown Обоснование: 
K7_reason = "\u0414\u0430\u043D\u043D\u044B\u0439 \u043F\u0430\u0440\u0430\u043C\u0435\u0442\u0440 \u0437\u0430\u0432\u0438\u0441\u0438\u0442 \u043E\u0442 \u0441\u043F\u043E\u0441\u043E\u0431\u0430 \u043C\u043E\u043D\u0435\u0442\u0438\u0437\u0430\u0446\u0438\u0438 \u0438 \u043F\u0440\u043E\u0447\u0438\u0445 \u0444\u0430\u043A\u0442\u043E\u0440\u043E\u0432, \u043F\u043E \u043A\u043E\u0442\u043E\u0440\u044B\u043C \u0435\u0449\u0451 \u043D\u0435 \u0431\u044B\u043B\u043E \u043F\u0440\u0438\u043D\u044F\u0442\u043E \u0440\u0435\u0448\u0435\u043D\u0438\u044F. \u0412 \u0447\u0430\u0441\u0442\u043D\u043E\u0441\u0442\u0438, \u0435\u0441\u043B\u0438 \u0432 \u0434\u043E\u0433\u043E\u0432\u043E\u0440\u0430\u0445 \u043E \u043C\u043E\u043D\u0435\u0442\u0438\u0437\u0430\u0446\u0438\u0438 \u043F\u043B\u0430\u0442\u0430 \u0431\u0443\u0434\u0435\u0442 \u043F\u0440\u043E\u0438\u0437\u0432\u043E\u0434\u0438\u0442\u044C\u0441\u044F \u043F\u043E \u0432\u0440\u0435\u043C\u0435\u043D\u0438, \u0441\u0431\u043E\u0438 \u0432 \u0440\u0430\u0431\u043E\u0442\u0435 \u043C\u043E\u0433\u0443\u0442 \u043F\u0440\u0438\u0432\u0435\u0441\u0442\u0438 \u043A \u0441\u0443\u0434\u0435\u0431\u043D\u043E\u043C\u0443 \u0440\u0430\u0437\u0431\u0438\u0440\u0430\u0442\u0435\u043B\u044C\u0441\u0442\u0432\u0443 \u0438 \u043F\u0440\u043E\u0447\u0438\u043C \u0441\u0438\u043B\u044C\u043D\u044B\u043C \u043D\u0435\u0433\u0430\u0442\u0438\u0432\u043D\u044B\u043C \u043F\u043E\u0441\u043B\u0435\u0434\u0441\u0442\u0432\u0438\u044F\u043C." #@param {type:"string"}

#@markdown ### K8 Восстанавливаемость 
#@markdown Среднее время восстановления работоспособности после отказа. Принимает следующие значения:
#@markdown - 1,12 - высокая 
#@markdown - 1 - средняя 
#@markdown - 0,98 - низкая 
K8 = 1 #@param ["1.12", "1", "0.98"]{type:"raw"}
#@markdown Обоснование: 
K8_reason = "\u0410\u043D\u0430\u043B\u043E\u0433\u0438\u0447\u043D\u043E \u043F\u0440\u0435\u0434\u044B\u0434\u0443\u0449\u0435\u043C\u0443" #@param {type:"string"}

#@markdown ### K9 Длительность обработки (время отклика)
#@markdown Быстрота реакции системы на входные воздействия. Принимает следующие значения:
#@markdown - 1,21 - быстрая 
#@markdown - 1 - умеренная
#@markdown - 0,92 - медленная
K9 = 1 #@param ["1.21", "1", "0.92"]{type:"raw"}
#@markdown Обоснование: 
K9_reason = "\u041A \u0441\u043E\u0436\u0430\u043B\u0435\u043D\u0438\u044E, \u043F\u043E \u043E\u043F\u0438\u0441\u0430\u043D\u0438\u044E \u043C\u0435\u0442\u043E\u0434\u0438\u043A\u0438 \u043D\u0435\u043F\u043E\u043D\u044F\u0442\u043D\u043E, \u0447\u0442\u043E \u0441\u0447\u0438\u0442\u0430\u0442\u044C \"\u0431\u044B\u0441\u0442\u0440\u044B\u043C\", \"\u0443\u043C\u0435\u0440\u0435\u043D\u043D\u044B\u043C\" \u0438 \u043F\u0440\u043E\u0447. \u041F\u043E\u044D\u0442\u043E\u043C\u0443 \u0434\u0430\u043D\u043D\u044B\u0439 \u043F\u0430\u0440\u0430\u043C\u0435\u0442\u0440 \u0442\u0430\u043A\u0436\u0435 \u043C\u043E\u0436\u0435\u0442 \u0431\u044B\u0442\u044C \u043F\u043E\u0434\u0432\u0435\u0440\u0436\u0435\u043D \u0438\u0437\u043C\u0435\u043D\u0435\u043D\u0438\u044E." #@param {type:"string"}

#@markdown ### K10 Исходный язык разработки ИС
#@markdown Определяется в зависимости от типа исходного языка, используемого при разработке ИС. Принимает следующие значения:
#@markdown - 1,08 - традиционный (Кобол, Фортран и т.д.)
#@markdown - 1,09 - процедурный (Си или эквивалентный)
#@markdown - 1,07 - функциональный (Лисп или эквивалентный)
#@markdown - 1 -объектноориентированный (Си++ или эквивалентный)
K10 = 1 #@param ["1.08", "1.09", "1.07", "1"]{type:"raw"}
#@markdown Обоснование: 
K10_reason = "\u0425\u043E\u0442\u044F Python \u043C\u043D\u043E\u0433\u0438\u0435 \u043C\u043E\u0433\u0443\u0442 \u043D\u0430\u0437\u0432\u0430\u0442\u044C \u0444\u0443\u043D\u043A\u0446\u0438\u043E\u043D\u0430\u043B\u044C\u043D\u044B\u043C \u044F\u0437\u044B\u043A\u043E\u043C, \u043E\u043D \u043F\u043E\u043B\u043D\u043E\u0441\u0442\u044C\u044E \u043F\u043E\u0434\u0434\u0435\u0440\u0436\u0438\u0432\u0430\u0435\u0442 \u043E\u0431\u044A\u0435\u043A\u0442\u044B." #@param {type:"string"}

#@markdown ## Факторы среды

#@markdown ### K11 Класс пользователя
#@markdown Определяется в зависимости от уровня мастерства или характеристик определенного класса пользователей. Пользователем может быть система, являющаяся внешней по отношении к рассматриваемой системе. Принимает следующие значения:
#@markdown - 1,12 - начинающий 
#@markdwon - 1,07 - средний 
#@markdown - 1 - специалист (эксперт)
#@markdown - 1,14 - случайный 
#@markdown - 1,06 - другая ИС (ПО) 
#@markdown - 1,09 - технические средства 
K11 = 1.14 #@param ["1.12", "1.07", "1", "1.14", "1.06", "1.09"]{type:"raw"}
#@markdown Обоснование: 
K11_reason = "\u0414\u0430\u043D\u043D\u044B\u0439 \u043F\u0430\u0440\u0430\u043C\u0435\u0442\u0440 \u0441\u043F\u043E\u0440\u043D\u044B\u0439, \u0447\u0442\u043E \u043C\u044B \u043F\u043E\u043D\u0438\u043C\u0430\u0435\u043C \u043F\u043E\u0434 \u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u0435\u043B\u0435\u043C? \u0412 \u043D\u0430\u0448\u0435\u0439 \u0442\u0440\u0430\u043A\u0442\u043E\u0432\u043A\u0435, \u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u0435\u043B\u044F\u043C\u0438 \u0431\u0443\u0434\u0443\u0442 \u0432\u0441\u0435 \u0430\u043A\u0442\u043E\u0440\u044B, \u0438 \u043E\u043D\u0438 \u0432\u0441\u0435 \u0440\u0430\u0437\u043D\u044B\u0435, \u043F\u043E\u044D\u0442\u043E\u043C\u0443 \u0441\u043B\u0443\u0447\u0430\u0439\u043D\u044B\u0439." #@param {type:"string"}

#@markdown ### K12 Требования к центральному обрабатывающему устройству (процессору)
#@markdown Определяются требованиями к тактовой частоте процессора (скорости процессора). Принимает следующие значения: 
#@markdown - 0,99 - высокая 
#@markdown - 1 - средняя
K12 = 1 #@param ["0.99", "1"]{type:"raw"}
#@markdown Обоснование: 
K12_reason = "\u0422\u0435\u0445\u043D\u0438\u0447\u0435\u0441\u043A\u0438\u0435 \u0442\u0440\u0435\u0431\u043E\u0432\u0430\u043D\u0438\u044F \u0443\u0441\u0442\u0430\u043D\u0430\u0432\u043B\u0438\u0432\u0430\u0435\u043C \u043F\u043E-\u043C\u0438\u043D\u0438\u043C\u0443\u043C\u0443, \u0442.\u043A. \u0432 \u0434\u0430\u043D\u043D\u043E\u043C \u043F\u043B\u0430\u043D\u0435 \u0435\u0449\u0451 \u043D\u0438\u0447\u0435\u0433\u043E \u043D\u0435 \u0440\u0435\u0448\u0435\u043D\u043E. \u041C\u043E\u0433\u0443\u0442 \u0431\u044B\u0442\u044C \u0438\u0437\u043C\u0435\u043D\u0435\u043D\u0438\u044F." #@param {type:"string"}

#@markdown ### K13 Требования к оперативной (основной) памяти
#@markdown ИС должна быть идентифицировано по требованиям, предъявляемым к оперативной памяти (объем, быстродействие). Принимает следующие значения:
#@markdown - 1 - большая
#@markdown - 1,04 - малая
K13 = 1 #@param ["1", "1.04"]{type:"raw"}
#@markdown Обоснование: 
K13_reason = "\u0422\u0435\u0445\u043D\u0438\u0447\u0435\u0441\u043A\u0438\u0435 \u0442\u0440\u0435\u0431\u043E\u0432\u0430\u043D\u0438\u044F \u0443\u0441\u0442\u0430\u043D\u0430\u0432\u043B\u0438\u0432\u0430\u0435\u043C \u043F\u043E-\u043C\u0438\u043D\u0438\u043C\u0443\u043C\u0443, \u0442.\u043A. \u0432 \u0434\u0430\u043D\u043D\u043E\u043C \u043F\u043B\u0430\u043D\u0435 \u0435\u0449\u0451 \u043D\u0438\u0447\u0435\u0433\u043E \u043D\u0435 \u0440\u0435\u0448\u0435\u043D\u043E. \u041C\u043E\u0433\u0443\u0442 \u0431\u044B\u0442\u044C \u0438\u0437\u043C\u0435\u043D\u0435\u043D\u0438\u044F." #@param {type:"string"}


#@markdown ### K14 Требования к внешней памяти
#@markdown ИС должна быть идентифицировано по требованиям, предъявляемым к внешней памяти (объем, быстродействие). Принимает следующие значения:
#@markdown - 1 - большая
#@markdown - 1,01 - малая 
K14 = 1 #@param ["1", "1.01"]{type:"raw"}
#@markdown Обоснование: 
K14_reason = "\u0422\u0435\u0445\u043D\u0438\u0447\u0435\u0441\u043A\u0438\u0435 \u0442\u0440\u0435\u0431\u043E\u0432\u0430\u043D\u0438\u044F \u0443\u0441\u0442\u0430\u043D\u0430\u0432\u043B\u0438\u0432\u0430\u0435\u043C \u043F\u043E-\u043C\u0438\u043D\u0438\u043C\u0443\u043C\u0443, \u0442.\u043A. \u0432 \u0434\u0430\u043D\u043D\u043E\u043C \u043F\u043B\u0430\u043D\u0435 \u0435\u0449\u0451 \u043D\u0438\u0447\u0435\u0433\u043E \u043D\u0435 \u0440\u0435\u0448\u0435\u043D\u043E. \u041C\u043E\u0433\u0443\u0442 \u0431\u044B\u0442\u044C \u0438\u0437\u043C\u0435\u043D\u0435\u043D\u0438\u044F." #@param {type:"string"}


#@markdown ### K15 Требования к локальной вычислительной сети
#@markdown ИС должна быть идентифицировано по требованиям, предъявляемым к ЛВС (пропускная способность, зашита информации в сети). Принимает следующие значения:
#@markdown - 1 - высокие требования
#@markdown - 1,02 - средние требования 
K15 = 1.02 #@param ["1", "1.02"]{type:"raw"}
#@markdown Обоснование: 
K15_reason = "\u0422\u0435\u0445\u043D\u0438\u0447\u0435\u0441\u043A\u0438\u0435 \u0442\u0440\u0435\u0431\u043E\u0432\u0430\u043D\u0438\u044F \u0443\u0441\u0442\u0430\u043D\u0430\u0432\u043B\u0438\u0432\u0430\u0435\u043C \u043F\u043E-\u043C\u0438\u043D\u0438\u043C\u0443\u043C\u0443, \u0442.\u043A. \u0432 \u0434\u0430\u043D\u043D\u043E\u043C \u043F\u043B\u0430\u043D\u0435 \u0435\u0449\u0451 \u043D\u0438\u0447\u0435\u0433\u043E \u043D\u0435 \u0440\u0435\u0448\u0435\u043D\u043E. \u041C\u043E\u0433\u0443\u0442 \u0431\u044B\u0442\u044C \u0438\u0437\u043C\u0435\u043D\u0435\u043D\u0438\u044F." #@param {type:"string"}

#@markdown ### K16 Критичность ИС
#@markdown Определяется в зависимости от уровня целостности продукции, с учетом конкретной методологии оценки. Принимает следующие значения:
#@markdown - 1,18 - человеческая жизнь
#@markdown - 1,16 - национальная безопасность
#@markdown - 1,13 - социальный хаос и паника 
#@markdown - 1 - организационная безопасность
K16 = 1 #@param ["1.18", "1.16", "1.13", "1"]{type:"raw"}
#@markdown Обоснование: 
K16_reason = "\u0415\u0441\u043B\u0438 \u0447\u0442\u043E-\u0442\u043E \u0441\u043B\u043E\u043C\u0430\u0435\u0442\u0441\u044F, \u043F\u043B\u043E\u0445\u043E \u0442\u043E\u043B\u044C\u043A\u043E \u043D\u0430\u043C." #@param {type:"string"}


#@markdown ### K17 Готовность
#@markdown Определяется в зависимости от типа имеющегося в наличии ППО. Принимает следующие значения:
#@markdown - 0,99 - наличие в готовом виде (есть альтернативные продукты)
#@markdown - 1 - общедоступная (известная методика)
#@markdown - 1,11 - заказное (методика заказчика специфическая) 
#@markdown - 1,09 - запатентованное (методика разработчика)
K17 = 0.99 #@param ["0.99", "1", "1.11","1.09"]{type:"raw"}
#@markdown Обоснование: 
K17_reason = "\u0422\u0430\u043A\u0438\u0445 \u0431\u043E\u0442\u043E\u0432 - \u043C\u043D\u043E\u0433\u043E, \u0432\u043E\u0437\u043C\u043E\u0436\u043D\u043E \u0434\u0430\u0436\u0435 \u0441 \u043E\u0442\u043A\u0440\u044B\u0442\u044B\u043C \u043A\u043E\u0434\u043E\u043C." #@param {type:"string"}



#@markdown ## Факторы данных

#@markdown ### K18 Представление данных
#@markdown Определяется в зависимости от элементов, типов и структур данных. Принимает следующие значения:
#@markdown - 1 - реляционный
#@markdown - 1 - индексируемый(иерархический)
#@markdown - 1,08 - сетевой
#@markdown - 1,09 - объектный
#@markdown - 0,95 - форматированный файл
K18 = 1 #@param ["1", "1.08", "1.09", "0.95"]{type:"raw"}
#@markdown Обоснование: 
K18_reason = "\u0411\u0443\u0434\u0435\u043C \u0438\u0441\u043F\u043E\u043B\u044C\u0437\u043E\u0432\u0430\u0442\u044C \u0440\u0435\u043B\u044F\u0446\u0438\u043E\u043D\u043D\u0443\u044E \u0431\u0430\u0437\u0443 \u0434\u0430\u043D\u043D\u044B\u0445 (\u043D-\u0440 MSSQL)" #@param {type:"string"}

k = [-1, 
     (K1,   K1_reason), 
     (K2,   K2_reason),
     (K3,   K3_reason),
     (K4,   K4_reason),
     (K5,   K5_reason),
     (K6,   K6_reason),
     (K7,   K7_reason),
     (K8,   K8_reason),
     (K9,   K9_reason),
     (K10, K10_reason), 
     (K11, K11_reason),
     (K12, K12_reason),
     (K13, K13_reason),
     (K14, K14_reason),
     (K15, K15_reason),
     (K16, K16_reason),
     (K17, K17_reason),
     (K18, K18_reason),
    ]

import pprint
pprint.pprint(k[1:])



[(1.05,
  'Поскольку запросы на получение контента должны быть обработаны как можно '
  'быстрее, будем считать, что ИС работает в режиме реального времени для '
  'обеспечения ответа данным запросам.'),
 (1.05,
  'К сожалению, маркетинговых исследований и прогнозов, какое количество '
  'аудитории мы сможем привлечь, пока не было произведено, данная оценка была '
  'сделана по принципу "пальцем в небо" и подвержена корректировке.'),
 (1,
  'Обновления являются достаточно вероятными событиями, но необходимость '
  'выполнять их постоянно отсутствует.'),
 (1.05,
  'На то есть несколько причин:  Необходимо защитить коммерческую информацию - '
  'про монетизацию. Поскольку проект имеет направленность на IT, имеется '
  'вероятность, что какой-нибудь хакер воспримет это как "вызов". Кто знает, '
  'какие отношения могут получиться с наёмной командой разработки...'),
 (1.03, 'Аналогично предыдущему'),
 (1.08,
  'Тотальный контроль над системой не требуется, но возможность отследить '
  'выб

$КП1 = К11 ∗ К16 ∗ К17$

$КП2 = К1 ∗ К2 ∗ К4 ∗ К5 ∗ К6 ∗ К7 ∗ К8 ∗ К9 ∗ К16 ∗ К17 ∗ К18$

$КП3 = К1 ∗ К2 ∗ К4 ∗ К5 ∗ К6 ∗ К7 ∗ К8 ∗ К9 ∗ К11 ∗ К12 ∗ К13 ∗ К14 ∗
К15 ∗ К16 ∗ К17 ∗ К18$

$КП4 = К1 ∗ К2 ∗ К4 ∗ К5 ∗ К6 ∗ К7 ∗ К8 ∗ К9 ∗ К10 ∗ К12 ∗ К13 ∗ К14 ∗
К15 ∗ К16 ∗ К17 ∗ К18$

$КП5 = К1 ∗ К2 ∗ К4 ∗ К5 ∗ К6 ∗ К7 ∗ К8 ∗ К9 ∗ К10 ∗ К11 ∗ К12 ∗ К13 ∗
К14 ∗ К15 ∗ К16 ∗ К17 ∗ К18$

$КП6 = К1 ∗ К2 ∗ К11 ∗ К16 ∗ К18$

In [0]:
#@title Расчёты

k_kp = [
    [11, 16, 17],
    [1,2,4,5,6,7,8,9,16,17,18],
    [1,2,4,5,6,7,8,9,11,12,13,14,15,16,17,18],
    [1,2,4,5,6,7,8,9,10,12,13,14,15,16,17,18],
    [1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18],
    [1,2,11,16,18]
]
kp = []
for l in k_kp:
    temp = 1
    for i in l:
        temp *= k[i][0]
    kp.append(temp)
    
pprint.pprint(kp)

[1.1285999999999998,
 1.2748646295000003,
 1.4824125911826003,
 1.3003619220900005,
 1.4824125911826003,
 1.25685]


## 4. Расчет трудоемкости разработки ПО с учетом поправочных коэффициентов

$S=КП_1*S_1+КП_2 * S_2+KП_3 * S_3+КП_4 * S_4+КП_5 *S_5+КП_6*S_6$

In [0]:
#@title Расчёты { display-mode: "form" }
S = np.sum(np.multiply(kp, S))
print('Трудоёмкость в человеко/месяцах:',S)

Трудоёмкость в человеко/месяцах: 58.140891959901104


# 5. Оценка сроков разработки ПО

In [0]:
#@title Расчёты { display-mode: "form" }
times = [
(5, 30),
(10, 80),
(17 , 140),
(26 , 210),
(37 , 280),
(50 , 340),
(65 , 400),
(80 , 450),
(100 , 500),
(120 , 550),
(140 , 610),
(160 , 670),
(180 , 720),
(200 , 770),
(230 , 820),
(260 , 870),
(290 , 930),
(330 , 990),
(370 , 1040),
(420 , 1090),
(470 , 1150),
(530 , 1200),
(600 , 1250),
(670 , 1300),
(750 , 1350),
(830 , 1400),
(900 , 1450),
(970 , 1500),
(1150 , 1550),
(1230 , 1600),
(1310 , 1660),
(1390 , 1720),
(1470 , 1780),
(1520 , 1840),
(1570 , 1900),
(1620 , 1960),
(1680 , 2020)
]

step = 40

answer_l = -1
if S >= times[-1][0] + step:
    answer_l = (S - times[-1][0])//step + len(times) + 1
else:
    for i, item in enumerate(times):
        if (item[0] >= S):
            answer_l = max(1,i)
            break

answer_r = -1
if S>= times[-1][1]:
    answer_l = (S - times[-1][1])//step + len(times) + 1
else:
    for i, item in enumerate(times):
        if (item[1]>S):
            answer_r = i + 1
            break

t = (answer_l + answer_r)//2

print('Длительность (месяцы)',t, ' (',answer_r,'-',answer_l,')')
        
    

Длительность (месяцы) 4  ( 2 - 6 )


In [0]:
#@title Корректировка трудоемкости при уменьшении срока разработки (при необходимости) { vertical-output: true, display-mode: "form" }
#@markdown **Процент сокращения**
X_percent = 34 #@param {type:"slider", min:0, max:100, step:1}

t_n = t*(100-X_percent)/100
print('Новый срок:',t_n)
if t_n < answer_r:
    print('Внимание! Новый срок меньше минимального:',answer_r)



#@markdown **Коэффициент эластичности трудоемкости**
L = 0.75 #@param {type:"number"}
S_cor = S*X_percent/100*L
print('Размер корректировки:', S_cor)
S -= S_cor
print('Новая трудоёмкость:', S)

Новый срок: 2.64
Размер корректировки: 0.49297088934915423
Новая трудоёмкость: 1.4402482845690974


# 6. Оценка стоимости ПО

In [0]:
#@title Расчёты { run: "auto", vertical-output: true }
#@markdown Средняя зарплата на текущий год
avg_wages_this_year = 50000 #@param {type:"integer"}
#@markdown Средняя инфляция
avg_inflation = 3 #@param {type:"slider", min:0, max:200, step:1}
#@markdown Социальные отчисления в процентах от зарплаты
soc_tax = 13 #@param {type:"slider", min:0, max:100, step:1}
#@markdown Накладные расходы в процентах от зарплаты
overhead = 71 #@param {type:"slider", min:0, max:100, step:1}
#@markdown Переодические расходы в процентах от зарплаты
period_cost = 48 #@param {type:"slider", min:0, max:100, step:1}
#@markdown Процент требуемой доходности
profitability = 10 #@param {type:"slider", min:0, max:100, step:1}
#@markdown НДС
vat = 20 #@param {type:"slider", min:0, max:100, step:1}
import math

r_sr = math.ceil(t/12)
print(r_sr)

avg_wage = [avg_wages_this_year]

for i in range(r_sr-1):
    temp = avg_wage[-1]*(1+avg_inflation/100)
    avg_wage.append(temp)
avg_costs = []

for i in range(r_sr):
    temp = avg_wage[i]
    temp *= 1 + (0.9*soc_tax + overhead + period_cost)/100
    temp *= 1 + profitability / 100
    temp += 1 + vat / 100
    avg_costs.append(temp)
Costs = sum(avg_costs)*S**r_sr/r_sr**r_sr
#@markdown Сколько знаков перед запятой исключаем
round_sign = 3#@param {type:"slider", min:0, max:6}
Costs = int(round(Costs, -round_sign))
print('Стоимость проекта:',Costs)

1
Стоимость проекта: 5883000


# 7. Оценка стоимости сопровождения

In [0]:
#@title Расчёты { run: "auto", vertical-output: true }
#@markdown Коэффициент трудоемкости сопровождения ППО
N_coef = 15 #@param {type:"slider", min:0, max:100, step:1}
s_Costs = Costs * N_coef / 100 * k[2][0] * k[3][0] * k[16][0]
#@markdown Сколько знаков перед запятой исключаем
round_sign = 3#@param {type:"slider", min:0, max:6}
s_Costs = int(round(s_Costs, -round_sign))
print('Стоимость сопровождения:',s_Costs)

Стоимость сопровождения: 927000


TODO: 
1. красивый экспорт результатов